In [325]:
import pandas as pd
import numpy as np
import re

import sys
sys.path.append('../')

import src.support as sp

In [326]:
originals = pd.read_csv("../data/netflix_originals.csv", index_col = 0)

In [327]:
originals.head()

,Title,Genre,Premiere,Runtime,IMDB Score,Language
0,Enter the Anime,Documentary,"August 5, 2019",58,2.5,English/Japanese
1,Dark Forces,Thriller,"August 21, 2020",81,2.6,Spanish
2,The App,Science fiction/Drama,"December 26, 2019",79,2.6,Italian
3,The Open House,Horror thriller,"January 19, 2018",94,3.2,English
4,Kaali Khuhi,Mystery,"October 30, 2020",90,3.4,Hindi


In [328]:
originals.shape

(584, 6)

In [329]:
df= pd.read_csv("../data/netflix_titles.csv")
df.head()

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...
3,s4,TV Show,Jailbirds New Orleans,NaN,NaN,NaN,"September 24, 2021",2021,TV-MA,1 Season,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo..."
4,s5,TV Show,Kota Factory,NaN,"Mayur More, Jitendra Kumar, Ranjan Raj, Alam K...",India,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...


In [330]:
df.shape

(8807, 12)

In [331]:
df.dtypes

show_id         object
type            object
title           object
director        object
cast            object
country         object
date_added      object
release_year     int64
rating          object
duration        object
listed_in       object
description     object
dtype: object

In [332]:
originals_df = df.merge(originals, left_on= "title", right_on="Title", how="inner")

In [333]:
originals_df.head(2)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description,Title,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm...",Dick Johnson Is Dead,Documentary,"October 2, 2020",90,7.5,English
1,s142,Movie,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada","September 1, 2021",2015,R,82 min,Action & Adventure,"When a retired CIA agent is kidnapped, his son...",Extraction,Action,"April 24, 2020",117,6.7,English


In [334]:
originals_df.drop(['release_year', 'rating', 'duration', 'listed_in', 'description','Title', "date_added"], axis = 1, inplace=True)

In [335]:
originals_df.head(2)

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,Documentary,"October 2, 2020",90,7.5,English
1,s142,Movie,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...","United States, United Kingdom, Canada",Action,"April 24, 2020",117,6.7,English


# Cleaning

## Country

In [336]:
originals_df.isnull().sum()

show_id        0
type           0
title          0
director      22
cast          91
country       14
Genre          0
Premiere       0
Runtime        0
IMDB Score     0
Language       0
dtype: int64

In [337]:
originals_df["country"] = originals_df["country"].apply(lambda x: x.split(",")[0] if type(x) == str else x)
originals_df.head(1)

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,Documentary,"October 2, 2020",90,7.5,English


## Language

In [338]:
originals_df["Language"] = originals_df["Language"].apply(lambda x: x.split("/")[0] if type(x) == str else x)
originals_df.head(2)

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,Documentary,"October 2, 2020",90,7.5,English
1,s142,Movie,Extraction,Steven C. Miller,"Bruce Willis, Kellan Lutz, Gina Carano, D.B. S...",United States,Action,"April 24, 2020",117,6.7,English


## Cast

In [339]:
originals_df["cast"] = originals_df["cast"].apply(lambda x: x.split(",") if type(x) == str else x)
originals_df = originals_df.explode("cast")

In [340]:
originals_df.head()

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,Documentary,"October 2, 2020",90,7.5,English
1,s142,Movie,Extraction,Steven C. Miller,Bruce Willis,United States,Action,"April 24, 2020",117,6.7,English
1,s142,Movie,Extraction,Steven C. Miller,Kellan Lutz,United States,Action,"April 24, 2020",117,6.7,English
1,s142,Movie,Extraction,Steven C. Miller,Gina Carano,United States,Action,"April 24, 2020",117,6.7,English
1,s142,Movie,Extraction,Steven C. Miller,D.B. Sweeney,United States,Action,"April 24, 2020",117,6.7,English


## Genre

In [342]:
originals_df["Genre"] = originals_df["Genre"].apply(lambda x: x.split("/")[0].split("-")[0].replace("film", "").replace("thriller", "").strip() if type(x) == str else x)
originals_df.head(2)

,show_id,type,title,director,cast,country,Genre,Premiere,Runtime,IMDB Score,Language
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,Documentary,"October 2, 2020",90,7.5,English
1,s142,Movie,Extraction,Steven C. Miller,Bruce Willis,United States,Action,"April 24, 2020",117,6.7,English


In [345]:
def clean_genre(x):
    
    drama = re.findall("drama", x)
    romantic = re.findall("Roman", x)
    dance = re.findall("[D|d]ance|[M|m]us",x)
    comedy = re.findall("[C|c]ome", x)
    nulos = re.findall("^$|One", x)

    if drama:
        return ""
    elif dance:
        return "Musical"
    elif romantic:
        return "Romantic"
    elif comedy:
        return "Comedy"
    
    elif nulos:
        return np.nan
    else:
        return x

    

In [346]:
originals_df["Genre"] = originals_df["Genre"].apply(clean_genre)

In [351]:
originals_df.to_csv("../data/netflix_originals_clean.csv")

In [352]:
originals_df["Genre"].unique()

array(['Documentary', 'Action', 'Thriller', 'Drama', 'Horror', 'Biopic',
       'Zombie', 'Comedy', '', 'Psychological', 'Science fiction',
       'Musical', 'Superhero', 'Romantic', 'Hidden', 'Animation',
       'Aftershow', 'Concert Film', 'Stop Motion', 'Family', 'Anthology',
       'Mystery', 'Variety show', 'Anime', 'Heist', 'Satire', 'Sports',
       'Political', 'Horror anthology', 'Adventure', 'Coming', 'Spy',
       'Science fiction adventure', nan, 'Christmas',
       'Psychological horror', 'Western', 'Historical',
       'Mentalism special', 'Biographical', 'Urban fantasy', 'Crime',
       'War', 'Mockumentary'], dtype=object)